In [ ]:
#pip install numpy pandas scikit-learn tensorflow transformers nltk spacy flask



In [ ]:
#pip install transformers

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('combined_amazon_reviews.csv')

# Display the first few rows
print(data.head())

# Check for missing values
print(data.isnull().sum())


                     id             dateAdded           dateUpdated  \
0  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
1  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
2  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
3  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
4  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   

                                                name                  asins  \
0  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
1  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
2  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
3  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
4  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   

          brand                                         categories  \
0  Amazonbasics  AA,AAA,Heal

C:\Users\diego\AppData\Local\Temp\ipykernel_10076\3521483802.py:4: DtypeWarning: Columns (1,2,3,7,8,11,14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('combined_amazon_reviews.csv')


In [6]:
print(data.columns)

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs',
       'reviews.dateAdded', 'reviews.userCity', 'reviews.userProvince'],
      dtype='object')


In [19]:
# Drop rows with missing values in relevant columns
data.dropna(subset=['reviews.text'], inplace=True)

# Clean review text if necessary (e.g., removing special characters)
import re
data['cleaned_review'] = data['reviews.text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()))


In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

inputs = tokenizer(data['cleaned_review'].tolist(), return_tensors='tf', padding=True, truncation=True)
labels = tf.convert_to_tensor(data['sentiment'].values)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(inputs['input_ids'], labels, epochs=3)


In [9]:
def sentiment_category(rating):
    if rating >= 4:
        return 'positive'
    elif rating <= 2:
        return 'negative'
    else:
        return 'neutral'

data['sentiment'] = data['reviews.rating'].apply(sentiment_category)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['cleaned_review'])
y = data['sentiment']


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negative       0.75      0.47      0.58       482
     neutral       0.59      0.11      0.18       603
    positive       0.94      0.99      0.97     12514

    accuracy                           0.94     13599
   macro avg       0.76      0.52      0.58     13599
weighted avg       0.92      0.94      0.92     13599



In [12]:
from transformers import BartTokenizer, TFBartForConditionalGeneration

# Load summarization model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Tokenize and summarize a review
inputs = tokenizer(data['cleaned_review'][0], return_tensors='tf', max_length=512, truncation=True)
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

C:\Users\diego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\diego\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

C:\Users\diego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


i order  of them and one of the item is bad quality is missing backup spring so i have to put a pcs of aluminum to make the battery work.


In [13]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your reviews are in 'cleaned_review'
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000)
X = vectorizer.fit_transform(data['cleaned_review'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from sklearn.decomposition import LatentDirichletAllocation

# Specify the number of topics (meta-categories)
n_topics = 6

lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# Get the top words for each topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

print_top_words(lda, vectorizer.get_feature_names_out(), 10)


Topic #0:
easy use tablet kids love great old friendly bought loves
Topic #1:
kindle tablet great reading read easy love books screen use
Topic #2:
loves tablet bought old gift great year games daughter christmas
Topic #3:
batteries good great price work value long last product well
Topic #4:
echo alexa tv music amazon great love use fire sound
Topic #5:
one bought buy best great kindle would ok product tablet


In [15]:
from sklearn.cluster import KMeans

# Fit KMeans with the number of desired clusters (meta-categories)
kmeans = KMeans(n_clusters=n_topics, random_state=42)
kmeans.fit(X)

# Assign each review to a cluster
data['category'] = kmeans.labels_
